In [1]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None),
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Import Data

In [3]:
df = pd.read_csv('../data/online_shoppers_intention.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Administrative           12330 non-null  int64  
 1   Administrative_Duration  12330 non-null  float64
 2   Informational            12330 non-null  int64  
 3   Informational_Duration   12330 non-null  float64
 4   ProductRelated           12330 non-null  int64  
 5   ProductRelated_Duration  12330 non-null  float64
 6   BounceRates              12330 non-null  float64
 7   ExitRates                12330 non-null  float64
 8   PageValues               12330 non-null  float64
 9   SpecialDay               12330 non-null  float64
 10  Month                    12330 non-null  object 
 11  OperatingSystems         12330 non-null  int64  
 12  Browser                  12330 non-null  int64  
 13  Region                   12330 non-null  int64  
 14  TrafficType           

In [5]:
# check target variable distribution
df.Revenue.value_counts()

False    10422
True      1908
Name: Revenue, dtype: int64

The target variable is highly unbalanced, so we will have to use some pre-processing techniques that will hopefully provide us with better performance in the machine learning models.
  
Helpful Links: 
* https://machinelearningmastery.com/what-is-imbalanced-classification/  
* https://machinelearningmastery.com/framework-for-imbalanced-classification-projects/


In [6]:
# Check for null values
df.isnull().sum()

Administrative             0
Administrative_Duration    0
Informational              0
Informational_Duration     0
ProductRelated             0
ProductRelated_Duration    0
BounceRates                0
ExitRates                  0
PageValues                 0
SpecialDay                 0
Month                      0
OperatingSystems           0
Browser                    0
Region                     0
TrafficType                0
VisitorType                0
Weekend                    0
Revenue                    0
dtype: int64

In [7]:
# force column names to lowercase
df.columns = df.columns.str.lower()

There are no null values in the dataframe. 

### Next Steps for machine learning preparation -- 
1) determine with columns are categorical, and 
2) which columns are numeric. 

In [8]:
df.sample(5)

,administrative,administrative_duration,informational,informational_duration,productrelated,productrelated_duration,bouncerates,exitrates,pagevalues,specialday,month,operatingsystems,browser,region,traffictype,visitortype,weekend,revenue
11494,1,68.50,0,0.00,11,372.82,0.00,0.04,54.77,0.00,Dec,1,1,1,2,Returning_Visitor,False,True
8099,0,0.00,0,0.00,20,1175.49,0.00,0.02,0.00,0.00,Dec,3,2,3,13,Returning_Visitor,False,False
4005,0,0.00,0,0.00,8,376.33,0.00,0.03,53.99,0.00,May,2,2,2,4,Returning_Visitor,True,True
11702,5,66.50,0,0.00,29,1141.17,0.01,0.03,0.00,0.00,Nov,3,2,1,2,Returning_Visitor,False,False
7968,2,103.30,0,0.00,38,1649.77,0.00,0.01,0.00,0.00,Oct,1,1,3,3,Returning_Visitor,False,False


According to the UCI dataset page, the dataset structure is as follows:

**Categorical:**
* `month`
* `operatingsystems`
* `browser`
* `region`
* `traffictype`
* `visitortype`
* `weekend`
* `revenue` (Target Feature)
  
**Numeric:**
* `administrative` - number of admin pages visited
* `informational`
* `productrelated`
* `administrative_duration`
* `informational_duration`
* `productrelated_duration`
* `bouncerates` (percent)
* `exitrates` (percent)
* `pagevalues` - definition somewhat ambigous
* `specialday` - numeric representation of proximity to holiday


In [9]:
# convert desired fields to categorical
df[['month','operatingsystems','browser','region','traffictype','visitortype','weekend','revenue']] = df[['month','operatingsystems','browser','region','traffictype','visitortype','weekend','revenue']].apply(lambda x: x.astype('category'))

In [10]:
# convert some fields to integers
df[['administrative','informational','productrelated']] = df[['administrative','informational','productrelated']].apply(lambda x: x.astype('int'))

# convert remaining fields to floats
df[['administrative_duration','informational_duration','productrelated_duration','bouncerates','exitrates','pagevalues','specialday']] = df[['administrative_duration','informational_duration','productrelated_duration','bouncerates','exitrates','pagevalues','specialday']].apply(lambda x: x.astype('float'))

In [11]:
# verify output matches desired transformations
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   administrative           12330 non-null  int32   
 1   administrative_duration  12330 non-null  float64 
 2   informational            12330 non-null  int32   
 3   informational_duration   12330 non-null  float64 
 4   productrelated           12330 non-null  int32   
 5   productrelated_duration  12330 non-null  float64 
 6   bouncerates              12330 non-null  float64 
 7   exitrates                12330 non-null  float64 
 8   pagevalues               12330 non-null  float64 
 9   specialday               12330 non-null  float64 
 10  month                    12330 non-null  category
 11  operatingsystems         12330 non-null  category
 12  browser                  12330 non-null  category
 13  region                   12330 non-null  category
 14  traffi

In [12]:
df.sample(5)

,administrative,administrative_duration,informational,informational_duration,productrelated,productrelated_duration,bouncerates,exitrates,pagevalues,specialday,month,operatingsystems,browser,region,traffictype,visitortype,weekend,revenue
128,4,462.00,0,0.00,51,1873.22,0.00,0.01,0.00,0.60,Feb,2,2,9,3,Returning_Visitor,False,False
11166,0,0.00,0,0.00,1,0.00,0.20,0.20,0.00,0.00,Dec,3,2,1,2,Returning_Visitor,False,False
1943,0,0.00,0,0.00,2,18.00,0.10,0.15,0.00,0.00,Mar,2,5,3,10,Returning_Visitor,False,False
7341,5,95.10,4,116.80,53,1696.57,0.00,0.02,0.00,0.00,Oct,2,2,5,2,Returning_Visitor,False,False
2097,0,0.00,0,0.00,1,0.00,0.20,0.20,0.00,0.00,May,1,1,1,3,Returning_Visitor,False,False


## Export Coerced CSV to pikl for datatype retention

In [15]:
df.to_pickle('../data/df_cleaned.pkl')